## topic 분류 확인하기 

#### [진행 계획]
1. data 불러오기 
2. 전처리 진행 
    - 필요없는 clounms 삭제하기 
3. 분류된 topic별로 내용이랑 topic 관련성 확인하기 

In [40]:
import pandas as pd
import numpy as np
import os
import glob
from time import perf_counter
import tqdm 
import matplotlib.pyplot as plt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
df = pd.read_csv("/aiffel/PocLab/meta_data_included_raw.csv")

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_2 = df.copy()

In [4]:
df

Unnamed: 0         title creator distributor  year  \
0                 0  DZJD21002058    솔트룩스        솔트룩스  2021   
1                 1  DZJD21002058    솔트룩스        솔트룩스  2021   
2                 2  DZJD21002058    솔트룩스        솔트룩스  2021   
3                 3  DZJD21002058    솔트룩스        솔트룩스  2021   
4                 4  DZJD21002058    솔트룩스        솔트룩스  2021   
...             ...           ...     ...         ...   ...   
2163205      296488  DJSX20004422    솔트룩스        솔트룩스  2020   
2163206      296489  DJSX20004422    솔트룩스        솔트룩스  2020   
2163207      296490  DJSX20004422    솔트룩스        솔트룩스  2020   
2163208      296491  DJSX20004422    솔트룩스        솔트룩스  2020   
2163209      296492  DJSX20004422    솔트룩스        솔트룩스  2020   

                      category annotation_level sampling  author publisher  \
0         제주방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
1         제주방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
2         제주방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
3         제주방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
4         제주방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
...                        ...              ...      ...     ...       ...   
2163205  전라 방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
2163206  전라 방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
2163207  전라 방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
2163208  전라 방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   
2163209  전라 방언 > 사적 대화 > 일상 대화           ['원시']    본문 전체  개인 발화자  개인 발화 녹음   

         ... prin_res curr_res age2 sex2 prin_res2 curr_res2  \
0        ...       제주       서울  30대   여성        제주        제주   
1        ...       제주       서울  30대   여성        제주        제주   
2        ...       제주       서울  30대   여성        제주        제주   
3        ...       제주       서울  30대   여성        제주        제주   
4        ...       제주       서울  30대   여성        제주        제주   
...      ...      ...      ...  ...  ...       ...       ...   
2163205  ...       광주       광주  40대   여성        광주        광주   
2163206  ...       광주       광주  40대   여성        광주        광주   
2163207  ...       광주       광주  40대   여성        광주        광주   
2163208  ...       광주       광주  40대   여성        광주        광주   
2163209  ...       광주       광주  40대   여성        광주        광주   

                                                      stdn  \
0        그냥 뭐랄까 우리 우리나라는 술을 이렇게 즐기면서 마신다기 보다는 분위기 때문에 마...   
1                     거의 뭐 집에 마시는 애들도 집에서 혼술하거나 약간 그런 거 같고   
2                              아니 어렸을 때는 어떻게 그렇게 마셨는지 모르겠어   
3                         그렇게하니까 아 진짜 술을 많이 안 마셔야겠다 생각이 들어   
4                                        그런데 어려보여 그렇게 안 보여   
...                                                    ...   
2163205        좀 가면 그~ 뭐~ 신간도서라 좀 읽고 바로 팔잖아 그럼 돈이 좀 많이 나와.   
2163206                 얼마 전에도 갔더니 만 삼천 원 정도 나왔더라고 책 세 권에?   
2163207   그래가지고 열심히 우리 신랑은 사가지고 이거 -읽- 읽는데 난 지금 열심히 팔고 있지.   
2163208                                 일기 형식으로 나온느 시집 있지.   
2163209  그 당시에 그~ 맞는 뭐~ 요즘들 보면 웹툰 작가처럼 말을 되게 이쁘게 하는 사람들...   

                                                      dial  \
0        그냥 뭐랄까 우리 우리나라는 술을 이렇게 즐기면서 마신다기 보다는 분위기 때문에 마...   
1                     거의 뭐 집에 마시는 애들도 집에서 혼술하거나 약간 그런 거 닮고   
2                                 아니 어렸을 때는 어떵 겅 마셔신지 모르크라   
3                           경하니까 아 진짜 술을 많이 안 마셔야겠다 생각이 들어   
4                                      그런데 어려보염신게 그렇게 안 보여   
...                                                    ...   
2163205        쫌 가면 그~ 뭐~ 신간도서라 쫌 읽고 바로 팔잖아 그럼 돈이 쫌 많이 나와.   
2163206                 얼마 전에도 갔더니 만 삼천 원 정도 나왔드라고 책 세 권에?   
2163207   그래아지고 열심히 우리 신랑은 사가지고 이거 -읽- 읽는데 난 지금 열심히 팔고 있지.   
2163208                                 일기 형식으로 나온느 시집 있제.   
2163209  그 당시에 그~ 맞는 뭐~ 요즘들 보면 웹툰 작가처럼 말을 되게 이쁘게 하는 사람들...   

                                               eojeol_st

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2163210 entries, 0 to 2163209
Data columns (total 24 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Unnamed: 0        int64 
 1   title             object
 2   creator           object
 3   distributor       object
 4   year              object
 5   category          object
 6   annotation_level  object
 7   sampling          object
 8   author            object
 9   publisher         object
 10  date              object
 11  topic             object
 12  age               object
 13  sex               object
 14  prin_res          object
 15  curr_res          object
 16  age2              object
 17  sex2              object
 18  prin_res2         object
 19  curr_res2         object
 20  stdn              object
 21  dial              object
 22  eojeol_stdn       object
 23  reg               object
dtypes: int64(1), object(23)
memory usage: 396.1+ MB


#### drop한 부분

In [6]:
df.drop(columns=['title','creator','distributor'], inplace=True) 
# 필요 없어보여서 미리 drop했는데 복사본으로 다시 확인해봄 

In [7]:
print(df_2['title'].unique()) # drop하기 

['DZJD21002058' 'DZJD20001739' 'DZES20000586' ... 'DJSX20001463'
 'DJSX20003162' 'DJSX20004422']


In [8]:
print(df_2['creator'].unique()) # drop하기 

['솔트룩스' ' 솔트룩스' '솔트룩스 ' '코아인포메이션' '솔트록스' '정현찬(duffkek86@naver.com)' ' 트룩스'
 '이이은(sosoyou78@gmail.com)' '유미나(gurm0919@nate.com)' '인피닉' '디큐' '타임소프트'
 '도담에스엘' '도담에스엘 ' '2 김혜은(curator25@gwangju.com)' '솙트룩스'
 '3 조정인(curator36@gwangju.com)' '솔트륵스' '솥트룩스'
 '2 박소연(manager2@gwangju.com)' '2 강영란(curator55@gwangju.com)'
 '4 유혜미(curator70@gwangju.com)' ' 솔트룩스 ' '솔틀룩스'
 '2 김지우(curator22@gwangju.com)' '도담에스엘)' '2 조서연(curator35@gwangju.com)'
 '1 안 진(curator15@gwangju.com)' '1 배도협(curator18@gwangju.com)' '솔트룰스'
 '솥르룩스']


In [9]:
print(df_2['distributor'].unique()) # drop하기 

['솔트룩스' ' 솔트룩스' '솔트룩스 ' '코아인포메이션' '솔트록스' nan '솔트룩수' '인피닉' '디큐' '타임소프트'
 '도담에스엘' '솔틀룩스' '솔트룰스' '속트룩스' '솥트룩스' '솙트룩스' ' 솔트룩스 ' '솔트루스'
 '전남방언 > 사적 대화 > 일상 대화' '솔트룩스솔트룩스']


In [10]:
print(df['annotation_level'].unique()) # drop하기 

["['원시']" "['']" "['원시 ']" "[' 원시']" "[' 원시 ']" '[]' "['본문 전체']"
 "['전라방언 > 사적 대화 > 일상 대화 원시']" "['대화 원시']" "['원시`']"
 "['전라방언 > 사적 대화 > 일상 대화원시']" "[' 전라방언 > 사적 대화 > 일상 대화 원시']"
 "['전라방언 > 사적 대화 > 일상 대화']" "['전라방언 > 사적대화 > 일상대화']" "['원시  ']" "['일시']"
 "['워시']"]


In [11]:
print(df['sampling'].unique()) # drop하기 

['본문 전체' ' 본문 전체' '본문 전체 ' '본문 녹음' '본문 전제' '본문 전쳅' '개인 발화자' '본문전체' '분문 전체'
 nan '원시' '본문 ㅈ너체' ' 본문전체' ' 본문 전체 ' '본문  전체' '발화 전체' '전체 본문' '전라방언' '원시 '
 '본문전체 ' '전체' '본문' '발화 전체 ' '본문 저체' '본문전체상']


In [12]:
print(df['author'].unique()) # drop하기 

['개인 발화자' ' 개인 발화자' '개인 발화자 ' '개인 발화자\t' '개인발화자' nan '개인 바로하자' '개인  발화자'
 ' 개인 발화자 ' '개인 발화' '개인 발화자  \t\t\t\t\t\t\t\t' '개일발화자' '개인 발화 녹음' '개별 발화자'
 '개인 밸화자' '개일 발화자']


In [13]:
print(df['publisher'].unique()) # drop하기 

['개인 발화 녹음' '개인 발화 녹음 ' ' 개인 발화 녹음' '개인 발화녹음' '개이 발화 녹음' nan ' 개인 발화 녹음 '
 '개인 발화자' '발화 녹음' '개인 발화 녹은' '개인 발화자 녹음 ' '개인 발화자 녹음' ' 개인 발화자 녹음'
 '개인 발화 녹음2793' '발화 녹음 ' '개인 발화 녹으뮤' '개일 발화 녹음' '개안 발화 녹음' '개인 음성 녹음'
 '개인 발롸 녹음' '개인 발화 녹으' '개인 발화 정보' '개인 발화  녹음' '개인 발환 녹음' '개인 발화 녹읍'
 '개인 발화녹음 ' '발인 발화 녹음' '개인발화녹음' '개인 발화 전체' '개인 바활 녹음' '개인발화 녹음'
 '개인 발화자 녹움' '게인 발화 녹음' '개인 발화 내용' '개인발화자 녹음' '개인 박화 녹음' '개인 발화 자' '발화녹음'
 '개인 발화 녹' '개일 발화 녹음 ' '개인 발화 녹음`']


In [14]:
df.drop(columns=['annotation_level','sampling','author','publisher'], inplace=True) 

In [15]:
df.drop(columns=['year','date'], inplace=True) 

In [16]:
df

Unnamed: 0               category  topic  age sex prin_res curr_res  \
0                 0   제주방언 > 사적 대화 > 일상 대화  꿈(목표)  30대  여성       제주       서울   
1                 1   제주방언 > 사적 대화 > 일상 대화  꿈(목표)  30대  여성       제주       서울   
2                 2   제주방언 > 사적 대화 > 일상 대화  꿈(목표)  30대  여성       제주       서울   
3                 3   제주방언 > 사적 대화 > 일상 대화  꿈(목표)  30대  여성       제주       서울   
4                 4   제주방언 > 사적 대화 > 일상 대화  꿈(목표)  30대  여성       제주       서울   
...             ...                    ...    ...  ...  ..      ...      ...   
2163205      296488  전라 방언 > 사적 대화 > 일상 대화     문학  40대  여성       광주       광주   
2163206      296489  전라 방언 > 사적 대화 > 일상 대화     문학  40대  여성       광주       광주   
2163207      296490  전라 방언 > 사적 대화 > 일상 대화     문학  40대  여성       광주       광주   
2163208      296491  전라 방언 > 사적 대화 > 일상 대화     문학  40대  여성       광주       광주   
2163209      296492  전라 방언 > 사적 대화 > 일상 대화     문학  40대  여성       광주       광주   

        age2 sex2 prin_res2 curr_res2  \
0        30대   여성        제주        제주   
1        30대   여성        제주        제주   
2        30대   여성        제주        제주   
3        30대   여성        제주        제주   
4        30대   여성        제주        제주   
...      ...  ...       ...       ...   
2163205  40대   여성        광주        광주   
2163206  40대   여성        광주        광주   
2163207  40대   여성        광주        광주   
2163208  40대   여성        광주        광주   
2163209  40대   여성        광주        광주   

                                                      stdn  \
0        그냥 뭐랄까 우리 우리나라는 술을 이렇게 즐기면서 마신다기 보다는 분위기 때문에 마...   
1                     거의 뭐 집에 마시는 애들도 집에서 혼술하거나 약간 그런 거 같고   
2                              아니 어렸을 때는 어떻게 그렇게 마셨는지 모르겠어   
3                         그렇게하니까 아 진짜 술을 많이 안 마셔야겠다 생각이 들어   
4                                        그런데 어려보여 그렇게 안 보여   
...                                                    ...   
2163205        좀 가면 그~ 뭐~ 신간도서라 좀 읽고 바로 팔잖아 그럼 돈이 좀 많이 나와.   
2163206                 얼마 전에도 갔더니 만 삼천 원 정도 나왔더라고 책 세 권에?   
2163207   그래가지고 열심히 우리 신랑은 사가지고 이거 -읽- 읽는데 난 지금 열심히 팔고 있지.   
2163208                                 일기 형식으로 나온느 시집 있지.   
2163209  그 당시에 그~ 맞는 뭐~ 요즘들 보면 웹툰 작가처럼 말을 되게 이쁘게 하는 사람들...   

                                                      dial  \
0        그냥 뭐랄까 우리 우리나라는 술을 이렇게 즐기면서 마신다기 보다는 분위기 때문에 마...   
1                     거의 뭐 집에 마시는 애들도 집에서 혼술하거나 약간 그런 거 닮고   
2                                 아니 어렸을 때는 어떵 겅 마셔신지 모르크라   
3                           경하니까 아 진짜 술을 많이 안 마셔야겠다 생각이 들어   
4                                      그런데 어려보염신게 그렇게 안 보여   
...                                                    ...   
2163205        쫌 가면 그~ 뭐~ 신간도서라 쫌 읽고 바로 팔잖아 그럼 돈이 쫌 많이 나와.   
2163206                 얼마 전에도 갔더니 만 삼천 원 정도 나왔드라고 책 세 권에?   
2163207   그래아지고 열심히 우리 신랑은 사가지고 이거 -읽- 읽는데 난 지금 열심히 팔고 있지.   
2163208                                 일기 형식으로 나온느 시집 있제.   
2163209  그 당시에 그~ 맞는 뭐~ 요즘들 보면 웹툰 작가처럼 말을 되게 이쁘게 하는 사람들...   

                                               eojeol_stdn reg  
0                                          [['(닮아)/(같아)']]  jj  
1                                          [['(닮고)/(같고)']]  jj  
2        [['(어떵)/(어떻게)', '(겅)/(그렇게)', '(마셔신지)/(마셨는지)', ...  jj  
3                                          [['(경)/(그렇게)']]  jj  
4                                    [['(어려보염신게)/(어려보여)']]  jj  
...                                                    ...  ..  
2163205                [['(쫌)/(좀)', '(쫌)/(좀)', '(쫌)/(좀)']]  jd  
2163206                              [['(나왔드라고)/(나왔더라고)']]  jd  
2163207                              [['(그래아지고)/(그래가지고)']]  jd  
2163208                                  [['(있제.)/(있지.)']]  jd  
2163209                                  [['(긍께)/(그러니까)']]  jd  

[2163210 rows x 15 columns]

#### drop 해야하는지 고민되는 부분

In [17]:
print(df['curr_res'].value_counts())

제주    1081326
대전     201833
광주     147500
부산     139032
강원     110881
서울      96035
대구      79053
전북      73951
전남      60401
울산      42131
경남      38927
충북      25905
경기      25857
충남      17600
경북      16451
인천       3574
세종       2753
Name: curr_res, dtype: int64


In [18]:
print(df['curr_res2'].value_counts())

제주    1085012
대전     204427
광주     144693
부산     134163
강원     114867
서울      84121
대구      79244
전북      73768
전남      64475
울산      41583
경남      38485
경기      31798
충북      23009
충남      19497
경북      13142
인천       6505
세종       4421
Name: curr_res2, dtype: int64


In [19]:
print(df['age'].value_counts())

20대       796380
50대       349754
30대       346498
60대 이상    332922
40대       268205
10대        69451
Name: age, dtype: int64


In [20]:
print(df['age2'].value_counts())

20대       782458
60대 이상    374347
50대       352508
30대       308376
40대       269656
10대        75865
Name: age2, dtype: int64


In [21]:
print(df['sex'].value_counts())

여성    1678858
남성     484352
Name: sex, dtype: int64


In [22]:
print(df['sex2'].value_counts())

여성    1683121
남성     480089
Name: sex2, dtype: int64


- age 와 age2의 관계성을 모르겠다. 
- sex 와 sex2 성별이 바뀌는게 무슨 의미인거지???

In [23]:
print(df['topic'].value_counts())

가족            294043
먹거리           218961
회사/학교         153751
여행지(국내/해외)    148183
건강            124144
연애/결혼         106049
취미             81519
아르바이트          57995
성격             55294
꿈(목표)          52869
방송/연예          52202
역사             52033
반려동물           46246
쇼핑             43103
영화             42092
다이어트           40382
종교             39885
스타일            38021
계절             36450
드라마            35632
재테크            33455
게임             32139
스포츠/레저         32059
선물             31640
군대             31627
전자기기           30408
음악             30316
자동차/오토바이       27726
날씨             25250
경제             25002
콤플렉스           21510
자격증            20646
만화             18644
SNS            16928
문학             16801
좌우명            12160
공연             11891
정치             11284
4차산업혁명          8498
법률              6372
Name: topic, dtype: int64


In [24]:
print(df['topic'].nunique())

40


In [25]:
df[df['topic'].str.contains('법률', na = False)]

Unnamed: 0              category topic  age sex prin_res curr_res  \
502821       502821  제주방언 > 사적 대화 > 일상 대화    법률  20대  여성       제주       제주   
502822       502822  제주방언 > 사적 대화 > 일상 대화    법률  20대  여성       제주       제주   
502823       502823  제주방언 > 사적 대화 > 일상 대화    법률  20대  여성       제주       제주   
502824       502824  제주방언 > 사적 대화 > 일상 대화    법률  20대  여성       제주       제주   
502825       502825  제주방언 > 사적 대화 > 일상 대화    법률  20대  여성       제주       제주   
...             ...                   ...   ...  ...  ..      ...      ...   
2145365      278648  전라방언 > 사적 대화 > 일상 대화    법률  40대  여성       광주       광주   
2145366      278649  전라방언 > 사적 대화 > 일상 대화    법률  40대  여성       광주       광주   
2145367      278650  전라방언 > 사적 대화 > 일상 대화    법률  40대  여성       광주       광주   
2145368      278651  전라방언 > 사적 대화 > 일상 대화    법률  40대  여성       광주       광주   
2145369      278652  전라방언 > 사적 대화 > 일상 대화    법률  40대  여성       광주       광주   

        age2 sex2 prin_res2 curr_res2  \
502821   20대   여성        제주        제주   
502822   20대   여성        제주        제주   
502823   20대   여성        제주        제주   
502824   20대   여성        제주        제주   
502825   20대   여성        제주        제주   
...      ...  ...       ...       ...   
2145365  40대   여성        광주        광주   
2145366  40대   여성        광주        광주   
2145367  40대   여성        광주        광주   
2145368  40대   여성        광주        광주   
2145369  40대   여성        광주        광주   

                                                    stdn  \
502821                   그거 그 그 막 말 걸고 그러면 무조건 좋아하면 안 된대   
502822                                      영 막 민대 민 다음에   
502823                                   근데 그런 적도 있대 막 그   
502824                                그거 그렇게 돈 뜯는 사람도 있대   
502825                                     그게 아니라 외국인이라서   
...                                                  ...   
2145365                  어~ 정말 어떻게 어렸을 때부터 그거를 시작 시작해가지고   
2145366                      다른 나라 같으면 선진국에서는 엄청 그런 게 언니   
2145367                       거세 화학적 거세 이런 거를 해버렸으면 좋겠어.   
2145368  그러니까 뭐~ 화학적 거세를 하든 어쨌든 간에 예를 들어서 엔번방처럼 그렇게 큰 아동   
2145369                                       나중에 자기가 인제   

                                                  dial  \
502821                 그거 그 그 막 말 걸고 그러면 무조건 좋아하면 안 된댄   
502822                                    영 막 민댄 민 다음에   
502823                                 근데 그런 적도 있댄 막 그   
502824                              그거 그렇게 돈 뜯는 사람도 있댄   
502825                                  그게 아니라 외국인이라부난   
...                                                ...   
2145365                어~ 정말 어뜨케 어렸을 때부터 그거를 시작 시작해가지고   
2145366                    따른 나라 같으면 선진국에서는 엄청 그런 게 언니   
2145367                     거세 화학적 거세 이런 거를 해브럿으면 좋겠어.   
2145368  긍께 뭐~ 화학적 거세를 하든 어쨌든 간에 예를 들어서 엔번방처럼 그렇게 큰 아동   
2145369                                     나중에 자기가 인자   

                      eojeol_stdn reg  
502821            [['(된댄)/(된대)']]  jj  
502822            [['(민댄)/(민대)']]  jj  
502823            [['(있댄)/(있대)']]  jj  
502824            [['(있댄)/(있대)']]  jj  
502825   [['(외국인이라부난)/(외국인이라서)']]  jj  
...                           ...  ..  
2145365         [['(어뜨케)/(어떻게)']]  jd  
2145366           [['(따른)/(다른)']]  jd  
2145367     [['(해브럿으면)/(해버렸으면)']]  jd  
2145368         [['(긍께)/(그러니까)']]  jd  
2145369           [['(인자)/(인제)']]  jd  

[6372 rows x 15 columns]

In [26]:
print(df[['category']].value_counts())

category             
제주방언 > 사적 대화 > 일상 대화     1166385
경상방언 > 사적 대화 > 일상 대화      334234
충청방언 > 사적 대화 > 일상 대화      242541
전라방언 > 사적 대화 > 일상 대화      154033
강원방언 > 사적 대화 > 일상 대화      111415
                          ...   
전라방언 > 사적대화 > 일상              12
충청방언  > 사적 대화 > 일상 대화         11
전라벙언>사적대화>일상대화                10
전라 방언 > 일상 대화 > 사적대화           9
전라방언> 사전 대화> 일상 대화             1
Length: 114, dtype: int64


In [27]:
df[['topic','stdn','dial']]

topic                                               stdn  \
0        꿈(목표)  그냥 뭐랄까 우리 우리나라는 술을 이렇게 즐기면서 마신다기 보다는 분위기 때문에 마...   
1        꿈(목표)               거의 뭐 집에 마시는 애들도 집에서 혼술하거나 약간 그런 거 같고   
2        꿈(목표)                        아니 어렸을 때는 어떻게 그렇게 마셨는지 모르겠어   
3        꿈(목표)                   그렇게하니까 아 진짜 술을 많이 안 마셔야겠다 생각이 들어   
4        꿈(목표)                                  그런데 어려보여 그렇게 안 보여   
...        ...                                                ...   
2163205     문학        좀 가면 그~ 뭐~ 신간도서라 좀 읽고 바로 팔잖아 그럼 돈이 좀 많이 나와.   
2163206     문학                 얼마 전에도 갔더니 만 삼천 원 정도 나왔더라고 책 세 권에?   
2163207     문학   그래가지고 열심히 우리 신랑은 사가지고 이거 -읽- 읽는데 난 지금 열심히 팔고 있지.   
2163208     문학                                 일기 형식으로 나온느 시집 있지.   
2163209     문학  그 당시에 그~ 맞는 뭐~ 요즘들 보면 웹툰 작가처럼 말을 되게 이쁘게 하는 사람들...   

                                                      dial  
0        그냥 뭐랄까 우리 우리나라는 술을 이렇게 즐기면서 마신다기 보다는 분위기 때문에 마...  
1                     거의 뭐 집에 마시는 애들도 집에서 혼술하거나 약간 그런 거 닮고  
2                                 아니 어렸을 때는 어떵 겅 마셔신지 모르크라  
3                           경하니까 아 진짜 술을 많이 안 마셔야겠다 생각이 들어  
4                                      그런데 어려보염신게 그렇게 안 보여  
...                                                    ...  
2163205        쫌 가면 그~ 뭐~ 신간도서라 쫌 읽고 바로 팔잖아 그럼 돈이 쫌 많이 나와.  
2163206                 얼마 전에도 갔더니 만 삼천 원 정도 나왔드라고 책 세 권에?  
2163207   그래아지고 열심히 우리 신랑은 사가지고 이거 -읽- 읽는데 난 지금 열심히 팔고 있지.  
2163208                                 일기 형식으로 나온느 시집 있제.  
2163209  그 당시에 그~ 맞는 뭐~ 요즘들 보면 웹툰 작가처럼 말을 되게 이쁘게 하는 사람들...  

[2163210 rows x 3 columns]

In [28]:
# df[df['topic'].str.contains('꿈', na = False)]

In [29]:
dt_1 = df[df['topic'] == '가족']
dt_2 = df[df['topic'] == '먹거리']

In [30]:
fam = dt_1.sample(n=10, 
                         replace=False, 
                         weights=None, 
                         random_state=220, axis=0, ignore_index=True)
eat = dt_2.sample(n=10, 
                         replace=False, 
                         weights=None, 
                         random_state=220, axis=0, ignore_index=True)

In [31]:
fam[['topic','stdn','dial','reg']]

topic                                        stdn  \
0    가족              시각장애인이라도 여행도 갈 정도니까 막 건강하시네 #게   
1    가족                         아흔은 나니까 밥을 하면 어떻게 막   
2    가족            아들도 ((하도)) 일본 은행에 일본 은행에 합격 했어요.   
3    가족                      &name3&이 보면 항상 느끼는 거야.   
4    가족            그래서 저번에 새벽 한 시에 세탁기 돌리는 걸 녹음을 했어   
5    가족                             사춘기가 제대로 왔네 #게.   
6    가족                 이제 왜 비행기 이 십 사 개 월 부터 돈 받아?   
7    가족                        이리저리 거짓말 거기에 정 떨어졌어.   
8    가족                          서방이랑 그(시아방)/(시아버지)   
9    가족  좋겠어 언니는 얼굴이 평면이라 가지고 요새 티비도 평면이 ((유행이잖아.))   

                                        dial reg  
0              시각장애인이라도 여행도 갈 정도니까 막 건강하신게 게  jj  
1                          아흔은 나난 밥을 하믄 어떵 막  jj  
2           아들도 ((하도)) 일본 은행에 일본 은행에 합격 하연예.  jj  
3                      &name3&이 보믄 항상 느끼는 겨.  cc  
4            그래서 저번에 새벽 한 시에 세탁기 돌리는 걸 녹음을 핸  jj  
5                             사춘기가 제대로 와신 게.  jj  
6               이제 무사 비행기 이 십 사 개 월 부터 돈 받암샤  jj  
7                     이리저리 거지뿌래이 거게에 정 떨어졌어.  kw  
8                         서방이랑 그(시아방)/(시아버지)  jj  
9  좋겄어 언니는 얼굴이 평면이라 가주 요새 티비도 평면이 ((유행이잖아.))  cc

In [32]:
eat[['topic','stdn','dial','reg']]

topic                                  stdn  \
0   먹거리                               너 먹어봤어?   
1   먹거리  &name3& 너도 이번 주 생일이잖아 뭐 가지고 싶은 거 없어?   
2   먹거리   그걸로 이제 (()) 소금으로 인제 그~ 뭐~ 맛을 내는 거니까   
3   먹거리                                혼자 있었어   
4   먹거리          동짓날 놈이 집가는거 아니라고 하지 않았냐 어른들이   
5   먹거리                            요즘 조금해 요리.   
6   먹거리   안 먹을 수가 없잖아 그 봤는데두 좀 먹어 보니까는 맛은 있어요   
7   먹거리              맞다 우린 요즘 너무 자주 만났다 안 그래?   
8   먹거리         멸치다시다 #영 푹 해서 제주도 스타일로 막 해서 다   
9   먹거리           그 얘기 갑자기 왜 했지 아무튼 그거 지금은 없어   

                                  dial reg  
0                               너 먹어봔?  jj  
1  &name3& 너도 이번 주 생일이잖아 뭐 가지고 싶은 거 언?  jj  
2  그걸로 이제 (()) 소금으로 인제 그~ 뭐~ 맛을 내는 기니까  gs  
3                                혼자 있언  jj  
4         동짓날 놈이 집가는거 아니랜 하지 않애시냐 어른들이  jj  
5                           요즘 호꼼해 요리.  jj  
6  안 먹을 수가 없잖아 그 봤는데두 쫌 먹어 보니까는 맛은 있어요  cc  
7            맞다 우린 요즘 너무 자주 만났다 안 그러나?  kw  
8           멸치다시다 영 푹 행 제주도 스타일로 막 행 다  jj  
9           그 얘기 갑자기 왜 했지 아무튼 그거 지금은 언  jj

### model 

In [33]:
# model_ckpt = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
# model_ckpt2= 'Huffon/klue-roberta-base-nli'
model_ckpt3 ='ddobokki/klue-roberta-small-nli-sts'
# model_ckpt4 ='beomi/KcELECTRA-base-v2022'
# model_ckpt5 = 'lighthouse/mdeberta-v3-base-kor-further'
# model_ckpt6 ='klue/bert-base'
# model_ckpt7 = 'klue/roberta-small'

In [34]:
# ! pip install sentence_transformers

In [35]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer(model_ckpt3)

In [63]:
sentence = df.stdn.values
stdn = '오늘은 오빠랑 책을 사러 갔어요 '
k=3

In [64]:
def top_sts(stdn,sentence,model_ckpt3,k):
    k = k
    model = SentenceTransformer(model_ckpt3)
    start_time = perf_counter()
    embeddings = model.encode(sentence,convert_to_tensor=True)
    src_embeddings = model.encode(stdn,convert_to_tensor=True)
    top_k = np.argpartition(util.pytorch_cos_sim(src_embeddings, embeddings).to('cpu').numpy()[0],-k)[-k:]
    top_res = np.partition(util.pytorch_cos_sim(src_embeddings, embeddings).to('cpu').numpy()[0],-k)[-k:]
    latency = perf_counter() - start_time
    print(f'{latency * 1000:.3f} ms')
    return sentences[top_k], top_k, top_res

In [66]:
top_sts(stdn,sentence,model_ckpt3,k)

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 14.76 GiB total capacity; 7.89 GiB already allocated; 3.75 MiB free; 13.34 GiB reserved in total by PyTorch)

In [54]:
sentences = df.stdn.values
sentences_topic = df.topic.values
sentences_dial = df.dial.values

In [55]:
# cosine 유사도 threshold 기준을 넘는 문장 index 반환

def long_seq(stdn,topic,model_ckpt3,k, threshold):
    k = k
    model = SentenceTransformer(model_ckpt3)
    embeddings = model.encode(topic,convert_to_tensor=True)
    src_embeddings = model.encode(stdn,convert_to_tensor=True)
    top_k = np.argpartition(util.pytorch_cos_sim(src_embeddings, embeddings).to('cpu').numpy()[0],-k)[-k:]
    top_res = np.partition(util.pytorch_cos_sim(src_embeddings, embeddings).to('cpu').numpy()[0],-k)[-k:]
    mask = top_res > threshold
    top_k = top_k[mask]
    return top_k

In [22]:
# 문장 <sep 기준으로 합치기>
def create_long_text(text,sentences, sentences_eng, sentences_dial, model_ckpt, k,threshold):
    topk = long_seq(text,sentences,model_ckpt,k, threshold)
    sentences = '<sep>'.join(sentences[topk][::-1])
    eng_sentences = '<sep>'.join(sentences_eng[topk][::-1])
    dial_sentences ='<sep>'.join(sentences_dial[topk][::-1])
    
    return sentences, eng_sentences, dial_sentences

### ref
- https://velog.io/@ljs7463/%ED%8A%B9%EC%A0%95%EB%AC%B8%EC%9E%90%EA%B0%80-%ED%8F%AC%ED%95%A8%EB%90%98%EA%B1%B0%EB%82%98-%EC%9D%BC%EC%B9%98%ED%95%98%EB%8A%94-%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%96%89%EC%B6%94%EC%B6%9C%ED%95%98%EA%B8%B0 > 특정문자가 포함되거나 일치하는 데이터(행)추출하기

- https://jimmy-ai.tistory.com/226 > 특정 열 추출, 행 추출

- https://jimmy-ai.tistory.com/36 >loc , iloc 함수 